In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/29 14:04:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [29]:
import os
trip_file = 'fhvhv_tripdata_2020-03.csv'
directory = '/home/ubuntu/working/spark-examples/data/'
zone_file = 'taxi+_zone_lookup.csv'
trip_file_path = os.path.join(directory, trip_file)
zone_file_path = os.path.join(directory, zone_file)
trip_file_path

'/home/ubuntu/working/spark-examples/data/fhvhv_tripdata_2020-03.csv'

In [31]:
trip_data = spark.read.csv(f"file:///{trip_file_path}", inferSchema=True, header=True)
trip_data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [32]:
zone_data = spark.read.csv(f"{zone_file_path}", inferSchema=True, header=True)
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [33]:
query = """SELECT zone_data.Zone, count(*) AS trips
FROM trip_data JOIN zone_data ON trip_data.PULocationID = zone_data.LocationID
WHERE trip_data.hvfhs_license_num = 'HV0003'
GROUP BY zone_data.Zone 
order by trips desc"""

### 임시뷰를 만든다

In [34]:
zone_data.createOrReplaceTempView("zone_data")
trip_data.createOrReplaceTempView("trip_data")

In [35]:
spark.sql(
    """
    select * from trip_data
    limit 5;
"""
).show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [36]:
spark.sql(
    """
    select * from zone_data
    limit 5;
"""
).show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [37]:
spark.sql(query).show()

+--------------------+------+
|                Zone| trips|
+--------------------+------+
| Crown Heights North|163091|
|       East New York|134198|
|         JFK Airport|114179|
|        East Village|112017|
|      Bushwick South|110150|
|Central Harlem North|108070|
|   LaGuardia Airport|104119|
|Washington Height...| 97324|
|Flatbush/Ditmas Park| 95724|
|            Canarsie| 94484|
|TriBeCa/Civic Center| 94155|
|             Astoria| 92676|
|             Bedford| 90352|
|      Midtown Center| 90261|
|  Stuyvesant Heights| 88749|
|            Union Sq| 88372|
|Times Sq/Theatre ...| 86870|
|Prospect-Lefferts...| 84347|
|         Brownsville| 82764|
|Mott Haven/Port M...| 82396|
+--------------------+------+
only showing top 20 rows



In [39]:
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Sort ['trips DESC NULLS LAST], true
+- 'Aggregate ['zone_data.Zone], ['zone_data.Zone, 'count(1) AS trips#598]
   +- 'Filter ('trip_data.hvfhs_license_num = HV0003)
      +- 'Join Inner, ('trip_data.PULocationID = 'zone_data.LocationID)
         :- 'UnresolvedRelation [trip_data], [], false
         +- 'UnresolvedRelation [zone_data], [], false

== Analyzed Logical Plan ==
Zone: string, trips: bigint
Sort [trips#598L DESC NULLS LAST], true
+- Aggregate [Zone#483], [Zone#483, count(1) AS trips#598L]
   +- Filter (hvfhs_license_num#413 = HV0003)
      +- Join Inner, (PULocationID#417 = LocationID#481)
         :- SubqueryAlias trip_data
         :  +- View (`trip_data`, [hvfhs_license_num#413,dispatching_base_num#414,pickup_datetime#415,dropoff_datetime#416,PULocationID#417,DOLocationID#418,SR_Flag#419])
         :     +- Relation [hvfhs_license_num#413,dispatching_base_num#414,pickup_datetime#415,dropoff_datetime#416,PULocationID#417,DOLocationID#418,SR_Flag#4

In [38]:
spark.stop()